In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp dataset

<IPython.core.display.Javascript object>

# Dataset

In [ ]:
# hide
from nbdev.showdoc import *

<IPython.core.display.Javascript object>

In [ ]:
# export
import uuid
import numpy as np
import pandas as pd
from copy import deepcopy
import json
from pathlib import Path
from typing import Union, Tuple
import datetime as dt
from functools import wraps
from rich import print as rich_print
from typeguard import typechecked

<IPython.core.display.Javascript object>

### Considerations

Goal: Create dynamic Numerai dataset where we can add metadata and Numerai specific functionality while keeping the flexibility of Pandas DataFrames.

__Options:__
__1.__ Add metadata to DataFrame through `df.attrs['some_metadata'] = "some_metadata"`
1.1. Downside: [Not persistent with parquet](https://stackoverflow.com/questions/14688306/adding-meta-information-metadata-to-pandas-dataframe).

__2.__ Subclass from DataFrame and add functionality.
2.1. Cumbersome when used to using `pd.read_csv`, `pd.read_parquet`, etc.
2.2 More info: [StackOverflow](https://stackoverflow.com/questions/22155951/how-can-i-subclass-a-pandas-dataframe), [Pandas Docs](https://pandas.pydata.org/pandas-docs/stable/development/extending.html#extending-subclassing-pandas).

__3.__ Develop dedicated `Dataset` class on which DataFrame is an attribute (`.dataf`)
3.1. Easy to add functionality and typecheck.
3.2. Does not work out of the box with sklearn Transformers, but can be easily made compatible with a single decorator.
3.3. Easy to export and import metadata.


We adopt the convention:
 1. All feature column names should start with "feature".
 2. All target column names should start with "target".
 3. All prediction column names should start with "prediction".
 4. Every column for which this does not hold will be classified as an "aux column".

In [ ]:
#export
class Dataset:
    def __init__(self, dataf: pd.DataFrame, *args, **kwargs):
        self.dataf = dataf
        self.__dict__.update(*args, **kwargs)
        self.all_cols = list(self.dataf.columns)
        self.feature_cols = [col for col in self.all_cols if col.startswith("feature")]
        self.target_cols = [col for col in self.all_cols if col.startswith("target")]
        self.prediction_cols = [
            col for col in self.all_cols if col.startswith("prediction")
        ]
        self.not_aux_cols = self.feature_cols + self.target_cols + self.prediction_cols
        self.aux_cols = [
            col for col in self.all_cols if col not in self.not_aux_cols
        ]

    def copy_dataset(self):
        """Copy Dataset object"""
        return deepcopy(self)

    def copy_dataframe(self) -> pd.DataFrame:
        """Copy DataFrame part of Dataset"""
        return deepcopy(self.dataf)

    def export_json_metadata(self, file="config.json", verbose=False, **kwargs):
        """Export all attributes in Dataset that can be serialized to json file."""
        rich_print(f":file_folder: Exporting metadata to {file} :file_folder:")
        json_txt = json.dumps(
            self.__dict__, default=lambda o: "<not serializable>", **kwargs
        )
        if verbose:
            rich_print(json_txt)
        Path(file).write_text(json_txt)

    def import_json_metadata(self, file="config.json", verbose=False, **kwargs):
        """Load arbitrary data into Dataset object from json file"""
        rich_print(f":file_folder: Importing metadata from {file} :file_folder:")
        with open(file) as json_file:
            json_data = json.load(json_file, **kwargs)
        if verbose:
            rich_print(json_data)
        # Make sure there is no overwrite on DataFrame
        json_data.pop("dataf", None)
        self.__dict__.update(json_data)

    def get_column_selection(self, cols: Union[str, list]) -> pd.DataFrame:
        """Return DataFrame given selection of columns."""
        return self.dataf.loc[:, cols if isinstance(cols, list) else [cols]]

    @property
    def get_feature_data(self) -> pd.DataFrame:
        return self.get_column_selection(cols=self.feature_cols)

    @property
    def get_target_data(self) -> pd.DataFrame:
        return self.get_column_selection(cols=self.target_cols)

    @property
    def get_single_target_data(self) -> pd.DataFrame:
        return self.get_column_selection(cols=['target'])

    @property
    def get_prediction_data(self) -> pd.DataFrame:
        return self.get_column_selection(cols=self.prediction_cols)

    @property
    def get_aux_data(self) -> pd.DataFrame:
        """All columns that are not features, targets or predictions."""
        return self.get_column_selection(cols=self.aux_cols)

    def get_feature_target_pair(self, multi_target=False) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Get split of features and targets
        :param multi_target: Returns only 'target' column by default.
        Returns all target columns when set to True.
        """
        X = self.get_feature_data
        y = self.get_target_data if multi_target else self.get_single_target_data
        return X, y

    def __repr__(self) -> str:
        return f"Dataset of shape {self.dataf.shape}. Columns: {self.all_cols}"

    def __str__(self):
        return self.__repr__()

<IPython.core.display.Javascript object>

### Tests

In [ ]:
# Random DataFrame
test_features = [f"feature_{l}" for l in "ABCDEFGHIK"]
id_col = [uuid.uuid4().hex for _ in range(100)]

df = pd.DataFrame(np.random.uniform(size=(100, 10)), columns=test_features)
df["id"] = id_col
df[["target", "target_1", "target_2"]] = np.random.normal(size=(100, 3))
df["era"] = range(100)

<IPython.core.display.Javascript object>

In [ ]:
metadata = {
    "version": 2,
    "additional_info": "test_model",
    "multi_target": False,
    "tournament_type": "classic",
}
dataset = Dataset(df, metadata)

<IPython.core.display.Javascript object>

In [ ]:
dataset.dataf.head(2)

,feature_A,feature_B,feature_C,feature_D,feature_E,feature_F,feature_G,feature_H,feature_I,feature_K,id,target,target_1,target_2,era
0,0.578753,0.405128,0.047234,0.205910,0.637374,0.581116,0.860202,0.446304,0.172437,0.338446,a5eb28feb3f146c49150f697c1cce379,0.086451,-0.967327,2.649563,0
1,0.039875,0.506594,0.361053,0.999113,0.391599,0.839359,0.207021,0.728092,0.471376,0.244788,778f6c04a85c452b937061f1c43bf101,-0.358312,0.208280,-0.731115,1


<IPython.core.display.Javascript object>

In [ ]:
dataset.get_feature_data.head(2)

,feature_A,feature_B,feature_C,feature_D,feature_E,feature_F,feature_G,feature_H,feature_I,feature_K
0,0.578753,0.405128,0.047234,0.205910,0.637374,0.581116,0.860202,0.446304,0.172437,0.338446
1,0.039875,0.506594,0.361053,0.999113,0.391599,0.839359,0.207021,0.728092,0.471376,0.244788


<IPython.core.display.Javascript object>

In [ ]:
dataset.aux_cols

['id', 'era']

<IPython.core.display.Javascript object>

In [ ]:
dataset.get_aux_data.head(2)

,id,era
0,a5eb28feb3f146c49150f697c1cce379,0
1,778f6c04a85c452b937061f1c43bf101,1


<IPython.core.display.Javascript object>

In [ ]:
assert dataset.version == 2
assert dataset.multi_target == False

<IPython.core.display.Javascript object>

In [ ]:
dataset.export_json_metadata("config.json")

📁 Exporting metadata to config.json 📁

<IPython.core.display.Javascript object>

In [ ]:
dataset.import_json_metadata("config.json")

📁 Importing metadata from config.json 📁

<IPython.core.display.Javascript object>

In [ ]:
dataset.__dict__

{'dataf':     feature_A  feature_B  feature_C  feature_D  feature_E  feature_F  \
 0    0.578753   0.405128   0.047234   0.205910   0.637374   0.581116   
 1    0.039875   0.506594   0.361053   0.999113   0.391599   0.839359   
 2    0.103328   0.337968   0.446926   0.461889   0.754410   0.341543   
 3    0.972067   0.385811   0.293181   0.172918   0.159606   0.066749   
 4    0.466081   0.185161   0.030470   0.562352   0.449155   0.868348   
 ..        ...        ...        ...        ...        ...        ...   
 95   0.642678   0.902437   0.786457   0.826929   0.660843   0.634481   
 96   0.649477   0.461011   0.444455   0.982363   0.488417   0.469148   
 97   0.894097   0.419196   0.746081   0.542510   0.593814   0.407480   
 98   0.243970   0.974667   0.945914   0.066492   0.986008   0.380426   
 99   0.778692   0.304823   0.006070   0.164137   0.939141   0.765985   
 
     feature_G  feature_H  feature_I  feature_K  \
 0    0.860202   0.446304   0.172437   0.338446   
 1    0.207

<IPython.core.display.Javascript object>

In [ ]:
dataf2 = dataset.copy_dataframe()
assert dataf2.equals(dataset.dataf)

<IPython.core.display.Javascript object>

In [ ]:
dataset.get_target_data.head(2)

,target,target_1,target_2
0,0.086451,-0.967327,2.649563
1,-0.358312,0.208280,-0.731115


<IPython.core.display.Javascript object>

In [ ]:
dataset.get_single_target_data.head(2)

,target
0,0.086451
1,-0.358312


<IPython.core.display.Javascript object>

In [ ]:
dataset.dataf.loc[:, "prediction_test_1"] = np.random.uniform(size=len(dataset.dataf))
new_dataset = Dataset(dataset.dataf, dataset.__dict__)
assert new_dataset.prediction_cols == ["prediction_test_1"]
assert new_dataset.version == 2

<IPython.core.display.Javascript object>

In [ ]:
new_dataset.get_column_selection("id").head(2)

,id
0,a5eb28feb3f146c49150f697c1cce379
1,778f6c04a85c452b937061f1c43bf101


<IPython.core.display.Javascript object>

In [ ]:
new_dataset.get_column_selection(["id", "prediction_test_1"]).head(2)

,id,prediction_test_1
0,a5eb28feb3f146c49150f697c1cce379,0.817813
1,778f6c04a85c452b937061f1c43bf101,0.505252


<IPython.core.display.Javascript object>

In [ ]:
X, y = new_dataset.get_feature_target_pair(multi_target=False)


<IPython.core.display.Javascript object>

In [ ]:
X.head(2)

,feature_A,feature_B,feature_C,feature_D,feature_E,feature_F,feature_G,feature_H,feature_I,feature_K
0,0.578753,0.405128,0.047234,0.205910,0.637374,0.581116,0.860202,0.446304,0.172437,0.338446
1,0.039875,0.506594,0.361053,0.999113,0.391599,0.839359,0.207021,0.728092,0.471376,0.244788


<IPython.core.display.Javascript object>

In [ ]:
y.head(2)

,target
0,0.086451
1,-0.358312


<IPython.core.display.Javascript object>

In [ ]:
str(dataset)

"Dataset of shape (100, 16). Columns: ['feature_A', 'feature_B', 'feature_C', 'feature_D', 'feature_E', 'feature_F', 'feature_G', 'feature_H', 'feature_I', 'feature_K', 'id', 'target', 'target_1', 'target_2', 'era']"

<IPython.core.display.Javascript object>

-----------------------------------------------

In [ ]:
# hide
# Run this cell to sync all changes with library
from nbdev.export import notebook2script

notebook2script()

Converted 00_download.ipynb.
Converted 01_dataloaders.ipynb.
Converted 02_dataset.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04a_model.ipynb.
Converted 04b_modelpipeline.ipynb.
Converted 05_postprocessing.ipynb.
Converted 06_prediction_dataset.ipynb.
Converted 07_evaluation.ipynb.
Converted 08_key.ipynb.
Converted 09_submission.ipynb.
Converted 10_staker.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>